In [1]:
ls

Ibotta_Question_1.ipynb             customers.csv*
Ibotta_Question_2.ipynb             item_sample.csv
Test Collaborative Filtering.ipynb  item_sample.zip*
Test_PCA.ipynb                      lat_long_data.csv
brands.csv*                         retailers.csv*
categories.csv*                     top_related_sprite_customers.csv


In [2]:
import pandas as pd
import scipy.sparse as sparse
import numpy as np
from scipy.sparse.linalg import spsolve

In [3]:
ls

Ibotta_Question_1.ipynb             customers.csv*
Ibotta_Question_2.ipynb             item_sample.csv
Test Collaborative Filtering.ipynb  item_sample.zip*
Test_PCA.ipynb                      lat_long_data.csv
brands.csv*                         retailers.csv*
categories.csv*                     top_related_sprite_customers.csv


# Ibotta Test

A large number of product redemption offers are available through our mobile app to consumers. However, consumers do not see the majority of these offers. We need to assist consumers discover relevant offers.

Create a machine learning system which  recommends the top 10 product IDs for each user that they have not previously purchased, based on prior purchase behavior for that consumer.

● List any assumptions that you make, and provide clarity on the methodology you employ.
● Submit any details/documentation that will support your analysis.
● Identify additional data you would want before implementing the system into production.
● Provide guidance for implementation of the recommendation system.
● Provide any underlying analysis where possible, and present back your findings and a summary of
what you did and learned.

In [4]:
# Pull in item list and drop nulls
cleaned_retail = pd.read_csv("item_sample.csv").dropna()

In [5]:
cleaned_retail.head()

,customer_id,purchase_timestamp,category_id,brand_id,product_id,retailer_id
0,4995150,2016-05-19 10:42:06,71,3360,170212,12
1,4995150,2016-05-19 10:42:06,282,14043,438259,12
2,4995150,2016-05-19 10:42:06,282,13652,438932,12
3,4995150,2016-05-19 10:42:06,72,433,768289,12
4,8053153,2016-05-19 14:13:08,220,24854,388549,18


In [6]:
# Create an item lookup to use later for the recommender system
brand_product = cleaned_retail[['brand_id','product_id']]
item_lookup = pd.read_csv("brands.csv")
item_lookup.columns = ['brand_id','brand_name']
item_lookup = item_lookup.merge(brand_product, on='brand_id')[['product_id','brand_name' ]].drop_duplicates()
item_lookup['product_id'] = item_lookup.product_id.astype(str)

In [7]:
# Aggregate quantity bought by each customer
grouped_purchased = cleaned_retail.groupby(['customer_id', 'product_id'])['purchase_timestamp'].count().reset_index()
grouped_purchased.columns = ["customer_id", "product_id", "quantity"]
grouped_purchased.head()

,customer_id,product_id,quantity
0,156,8119,2
1,156,8120,1
2,156,8121,1
3,156,150209,2
4,156,174738,1


Instead of representing an explicit rating, the purchase quantity can represent a “confidence” in terms of how strong the interaction was. Items with a larger number of purchases by a customer can carry more weight in our ratings matrix of purchases.

Our last step is to create the sparse ratings matrix of users and items utilizing the code below:

In [8]:
customers = list(np.sort(grouped_purchased.customer_id.unique())) # Get our unique customers
products = list(np.sort(grouped_purchased.product_id.unique())) # Get our unique products that were purchased
quantity = list(np.sort(grouped_purchased.quantity)) # All of our purchases

rows = grouped_purchased.customer_id.astype('category', categories = customers).cat.codes 

# Get the associated row indices
cols = grouped_purchased.product_id.astype('category', categories = products).cat.codes

# Get the associated column indices
purchases_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape=(len(customers), len(products)))

In [9]:
# Check out sparse matrix object
purchases_sparse

<27055x114871 sparse matrix of type '<class 'numpy.int64'>'
	with 2239532 stored elements in Compressed Sparse Row format>

In [10]:
print ("customers:", len(customers), "products:", len(products))
print ("y:", len(purchases_sparse.getnnz(1)), "x:", len(purchases_sparse.getnnz(0)))

customers: 27055 products: 114871
y: 27055 x: 114871


In [11]:
# Check the sparsity of the matrix
matrix_size = purchases_sparse.shape[0]*purchases_sparse.shape[1] # Number of possible interactions in the matrix
num_purchases = len(purchases_sparse.nonzero()[0]) # Number of items interacted with
sparsity = 100*(1 - (num_purchases/matrix_size))
sparsity

99.9279391580165

99.9% of the interaction matrix is sparse. For collaborative filtering to work, the maximum sparsity you could get away with would probably be about 99.5% or so. We are above this, so we need to reduce the sparsity. 

Luckily we can easily remove rows and columns that have very low total counts.   Essentially removing low transcation items, and consumers. This is also related to "cold starting", a term used to described users without transaction history.  There are multiple ways of dealing with this within an application.

In [12]:
purchases_sparse = purchases_sparse[purchases_sparse.getnnz(1)>30][:,purchases_sparse.getnnz(0)>30]

matrix_size = purchases_sparse.shape[0]*purchases_sparse.shape[1] # Number of possible interactions in the matrix
num_purchases = len(purchases_sparse.nonzero()[0]) # Number of items interacted with
sparsity = 100*(1 - (num_purchases/matrix_size))
sparsity

99.41163625991679

In [70]:
[purchases_sparse.getnnz(1)>30]

[array([ True,  True,  True, ...,  True, False,  True], dtype=bool)]

In [13]:
print ("y:", len(purchases_sparse.getnnz(1)), "x:", len(purchases_sparse.getnnz(0)))

y: 15317 x: 16077


We chose to reduce the sparsity to around 99.3% just to provide a buffer between 99.5.
This new sparsity should work fine. 

# Creating a Training and Validation Set

Typically in Machine Learning applications, we need to test whether the model we just trained is any good on new data it hasn’t yet seen before from the training phase.

With collaborative filtering, that’s not going to work because you need all of the user/item interactions to find the proper matrix factorization. A better method is to hide a certain percentage of the user/item interactions from the model during the training phase chosen at random. Then, check during the test phase how many of the items that were recommended the user actually ended up purchasing in the end.

Ideally, you would ultimately test your recommendations with some kind of A/B test or utilizing data from a time series where all data prior to a certain point in time is used for training while data after a certain period of time is used for testing.

We will split 

In [14]:
import random
def make_train(ratings, pct_test = 0.2):
    '''
    This function will take in the original user-item matrix and "mask" a percentage of the original ratings where a
    user-item interaction has taken place for use as a test set. The test set will contain all of the original ratings, 
    while the training set replaces the specified percentage of them with a zero in the original ratings matrix. 
    
    parameters: 
    
    ratings - the original ratings matrix from which you want to generate a train/test set. Test is just a complete
    copy of the original set. This is in the form of a sparse csr_matrix. 
    
    pct_test - The percentage of user-item interactions where an interaction took place that you want to mask in the 
    training set for later comparison to the test set, which contains all of the original ratings. 
    
    returns:
    
    training_set - The altered version of the original data with a certain percentage of the user-item pairs 
    that originally had interaction set back to zero.
    
    test_set - A copy of the original ratings matrix, unaltered, so it can be used to see how the rank order 
    compares with the actual interactions.
    
    user_inds - From the randomly selected user-item indices, which user rows were altered in the training data.
    This will be necessary later when evaluating the performance via AUC.
    '''
    test_set = ratings.copy() # Make a copy of the original set to be the test set. 
    test_set[test_set != 0] = 1 # Store the test set as a binary preference matrix
    training_set = ratings.copy() # Make a copy of the original data we can alter as our training set. 
    nonzero_inds = training_set.nonzero() # Find the indices in the ratings data where an interaction exists
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1])) # Zip these pairs together of user,item index into list
    random.seed(0) # Set the random seed to zero for reproducibility
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs))) # Round the number of samples needed to the nearest integer
    samples = random.sample(nonzero_pairs, num_samples) # Sample a random number of user-item pairs without replacement
    user_inds = [index[0] for index in samples] # Get the user row indices
    item_inds = [index[1] for index in samples] # Get the item column indices
    training_set[user_inds, item_inds] = 0 # Assign all of the randomly chosen user-item pairs to zero
    training_set.eliminate_zeros() # Get rid of zeros in sparse array storage after update to save space
    return training_set, test_set, list(set(user_inds)) # Output the unique list of user rows that were altered  

In [15]:
product_train, product_test, product_users_altered = make_train(purchases_sparse, pct_test = 0.2)

Now that we have our train/test split, it is time to implement the alternating least squares algorithm from the Hu, Koren, and Volinsky paper.

In [16]:
def implicit_weighted_ALS(training_set, lambda_val = 0.1, alpha = 40, iterations = 10, rank_size = 20, seed = 0):
    '''
    Implicit weighted ALS taken from Hu, Koren, and Volinsky 2008. Designed for alternating least squares and implicit
    feedback based collaborative filtering. 
    
    parameters:
    
    training_set - Our matrix of ratings with shape m x n, where m is the number of users and n is the number of items.
    Should be a sparse csr matrix to save space. 
    
    lambda_val - Used for regularization during alternating least squares. Increasing this value may increase bias
    but decrease variance. Default is 0.1. 
    
    alpha - The parameter associated with the confidence matrix discussed in the paper, where Cui = 1 + alpha*Rui. 
    The paper found a default of 40 most effective. Decreasing this will decrease the variability in confidence between
    various ratings.
    
    iterations - The number of times to alternate between both user feature vector and item feature vector in
    alternating least squares. More iterations will allow better convergence at the cost of increased computation. 
    The authors found 10 iterations was sufficient, but more may be required to converge. 
    
    rank_size - The number of latent features in the user/item feature vectors. The paper recommends varying this 
    between 20-200. Increasing the number of features may overfit but could reduce bias. 
    
    seed - Set the seed for reproducible results
    
    returns:
    
    The feature vectors for users and items. The dot product of these feature vectors should give you the expected 
    "rating" at each point in your original matrix. 
    '''
    
    # first set up our confidence matrix
    
    conf = (alpha*training_set) # To allow the matrix to stay sparse, I will add one later when each row is taken 
                                # and converted to dense. 
    num_user = conf.shape[0]
    num_item = conf.shape[1] # Get the size of our original ratings matrix, m x n
    
    # initialize our X/Y feature vectors randomly with a set seed
    rstate = np.random.RandomState(seed)
    
    X = sparse.csr_matrix(rstate.normal(size = (num_user, rank_size))) # Random numbers in a m x rank shape
    Y = sparse.csr_matrix(rstate.normal(size = (num_item, rank_size))) # Normally this would be rank x n but we can 
                                                                 # transpose at the end. Makes calculation more simple.
    X_eye = sparse.eye(num_user)
    Y_eye = sparse.eye(num_item)
    lambda_eye = lambda_val * sparse.eye(rank_size) # Our regularization term lambda*I. 
    
    # We can compute this before iteration starts. 
    
    # Begin iterations
   
    for iter_step in range(iterations): # Iterate back and forth between solving X given fixed Y and vice versa
        # Compute yTy and xTx at beginning of each iteration to save computing time
        yTy = Y.T.dot(Y)
        xTx = X.T.dot(X)
        # Being iteration to solve for X based on fixed Y
        for u in range(num_user):
            conf_samp = conf[u,:].toarray() # Grab user row from confidence matrix and convert to dense
            pref = conf_samp.copy() 
            pref[pref != 0] = 1 # Create binarized preference vector 
            CuI = sparse.diags(conf_samp, [0]) # Get Cu - I term, don't need to subtract 1 since we never added it 
            yTCuIY = Y.T.dot(CuI).dot(Y) # This is the yT(Cu-I)Y term 
            yTCupu = Y.T.dot(CuI + Y_eye).dot(pref.T) # This is the yTCuPu term, where we add the eye back in
                                                      # Cu - I + I = Cu
            X[u] = spsolve(yTy + yTCuIY + lambda_eye, yTCupu) 
            # Solve for Xu = ((yTy + yT(Cu-I)Y + lambda*I)^-1)yTCuPu, equation 4 from the paper  
        # Begin iteration to solve for Y based on fixed X 
        for i in range(num_item):
            conf_samp = conf[:,i].T.toarray() # transpose to get it in row format and convert to dense
            pref = conf_samp.copy()
            pref[pref != 0] = 1 # Create binarized preference vector
            CiI = sparse.diags(conf_samp, [0]) # Get Ci - I term, don't need to subtract 1 since we never added it
            xTCiIX = X.T.dot(CiI).dot(X) # This is the xT(Cu-I)X term
            xTCiPi = X.T.dot(CiI + X_eye).dot(pref.T) # This is the xTCiPi term
            Y[i] = spsolve(xTx + xTCiIX + lambda_eye, xTCiPi)
            # Solve for Yi = ((xTx + xT(Cu-I)X) + lambda*I)^-1)xTCiPi, equation 5 from the paper
    # End iterations
    return X, Y.T # Transpose at the end to make up for not being transposed at the beginning. 
                         # Y needs to be rank x n. Keep these as separate matrices for scale reasons. 

In [17]:
user_vecs, item_vecs = implicit_weighted_ALS(product_train, lambda_val = 0.1, alpha = 15, iterations = 1,
                                            rank_size = 20)

In [18]:
# For first user, looking at items vectors (user dot items)
user_vecs[0,:].dot(item_vecs).toarray()[0,:5]

array([  1.32236293e-05,   8.72394397e-06,   1.24393145e-05,
        -5.50696131e-05,   2.72019200e-05])

We can do the same by utilizing Ben Frederickson code at Flipboard, he came out with a version of ALS for Python utilizing Cython and parallelizing the code among threads.

In [19]:
import implicit

In [20]:
alpha = 15
user_vecs, item_vecs = implicit.alternating_least_squares((product_train*alpha).astype('double'), 
                                                          factors=20, 
                                                          regularization = 0.1, 
                                                         iterations = 50)

This method is deprecated. Please use the AlternatingLeastSquares class instead


# Evaluating the Recommender System

In [21]:
from sklearn import metrics

In [22]:
def auc_score(predictions, test):
    '''
    This simple function will output the area under the curve using sklearn's metrics. 
    
    parameters:
    
    - predictions: your prediction output
    
    - test: the actual target result you are comparing to
    
    returns:
    
    - AUC (area under the Receiver Operating Characterisic curve)
    '''
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr, tpr)  

In [23]:
def calc_mean_auc(training_set, altered_users, predictions, test_set):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered. 
    
    parameters:
    
    training_set - The training set resulting from make_train, where a certain percentage of the original
    user/item interactions are reset to zero to hide them from the model 
    
    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
    These should be stored in a list, with user vectors as item zero and item vectors as item one. 
    
    altered_users - The indices of the users where at least one user/item pair was altered from make_train function
    
    test_set - The test set constucted earlier from make_train function
    
    
    
    returns:
    
    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''
    
    
    store_auc = [] # An empty list to store the AUC for each user that had an item removed from the training set
    popularity_auc = [] # To store popular AUC scores
    pop_items = np.array(test_set.sum(axis = 0)).reshape(-1) # Get sum of item iteractions to find most popular
    item_vecs = predictions[1]
    for user in altered_users: # Iterate through each user that had an item altered
        training_row = training_set[user,:].toarray().reshape(-1) # Get the training set row
        zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training 
        pop = pop_items[zero_inds] # Get the item popularity for our chosen items
        store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user and store
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score
    # End users iteration
    
    return float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc))  
   # Return the mean AUC rounded to three decimal places for both test and popularity benchmark

In [24]:
calc_mean_auc(product_train, product_users_altered, 
              [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)
# AUC for our recommender system

(0.843, 0.739)

We can see that our recommender system beat popularity. Our system has a mean AUC of 0.84, while the popular item benchmark had a lower benchmark of .74. Ideally we would setup a crossvalidation for this to increase the accuracy of the reccommender system. 

In [25]:
customers_arr = np.array(customers) # Array of customer IDs from the ratings matrix
products_arr = np.array(products) # Array of product IDs from the ratings matrix

In [39]:
def get_items_purchased(customer_id, mf_train, customers_list, products_list, item_lookup):
    '''
    This just tells me which items have been already purchased by a specific user in the training set. 
    
    parameters: 
    
    customer_id - Input the customer's id number that you want to see prior purchases of at least once
    
    mf_train - The initial ratings training set used (without weights applied)
    
    customers_list - The array of customers used in the ratings matrix
    
    products_list - The array of products used in the ratings matrix
    
    item_lookup - A simple pandas dataframe of the unique product ID/product descriptions available
    
    returns:
    
    A list of item IDs and item descriptions for a particular customer that were already purchased in the training set
    '''
    cust_ind = np.where(customers_list == customer_id)[0][0] # Returns the index row of our customer id
    purchased_ind = mf_train[cust_ind,:].nonzero()[1] # Get column indices of purchased items
    prod_codes = products_list[purchased_ind] # Get the stock codes for our purchased items
    return item_lookup.loc[item_lookup.product_id.isin(prod_codes)]

In [27]:
customers_arr[:5]

array([ 156,  159, 2150, 2151, 2155])

In [44]:
get_items_purchased(2150, product_train, customers_arr, products_arr, item_lookup)

,product_id,brand_name


In [50]:
cust_ind = np.where(customers_arr == 159)[0][0]
cust_ind

1

In [55]:
np.where(customers_arr == 159)[0][0]

1

In [61]:
product_train[1,:].nonzero()[1]

array([  100,   103,   104,   113,   131,   138,   144,   446,   468,
         478,   479,   534,   629,   630,   634,   799,   938,   968,
        1015,  1120,  1267,  1284,  1494,  1656,  1876,  1878,  1885,
        1911,  2117,  2119,  2347,  2753,  3425,  3617,  4064,  4439,
        4768,  4814,  4834,  4892,  4986,  4994,  5033,  5133,  5206,
        5403,  5417,  5418,  5443,  5476,  5497,  5953,  5959,  6092,
        6094,  6156,  6257,  6337,  6375,  6411,  6760,  6800,  7018,
        7128,  7138,  7240,  7290,  7298,  7561,  7832,  7932,  7940,
        7966,  8120,  8241,  8281,  8410,  8661,  8805,  8911,  9092,
        9208,  9248,  9533,  9782, 10305, 10351, 10412, 10498, 10701,
       10911, 11317, 11442, 11457, 12029, 12189, 12233, 12273, 12278,
       12388, 12529, 12590, 12715, 13224, 13225, 13353, 13355, 13498,
       13708, 13712, 13760, 13884, 13886, 13983, 14022, 14025, 14079,
       14114, 14284, 14310, 14336, 14431, 14433, 14465, 14563, 15172,
       15214, 15272,

In [64]:
prod_codes = products_arr[[  100,   103,   104,   113,   131,   138,   144,   446,   468,
         478,   479,   534,   629,   630,   634,   799,   938,   968,
        1015,  1120,  1267,  1284,  1494,  1656,  1876,  1878,  1885,
        1911,  2117,  2119,  2347,  2753,  3425,  3617,  4064,  4439,
        4768,  4814,  4834,  4892,  4986,  4994,  5033,  5133,  5206,
        5403,  5417,  5418,  5443,  5476,  5497,  5953,  5959,  6092,
        6094,  6156,  6257,  6337,  6375,  6411,  6760,  6800,  7018,
        7128,  7138,  7240,  7290,  7298,  7561,  7832,  7932,  7940,
        7966,  8120,  8241,  8281,  8410,  8661,  8805,  8911,  9092,
        9208,  9248,  9533,  9782, 10305, 10351, 10412, 10498, 10701,
       10911, 11317, 11442, 11457, 12029, 12189, 12233, 12273, 12278,
       12388, 12529, 12590, 12715, 13224, 13225, 13353, 13355, 13498,
       13708, 13712, 13760, 13884, 13886, 13983, 14022, 14025, 14079,
       14114, 14284, 14310, 14336, 14431, 14433, 14465, 14563, 15172,
       15214, 15272, 15312, 15386, 15391, 15469, 15471, 15488, 15499,
       15628, 15778, 15902, 15907, 15908, 15909]] 
prod_codes

array([  343,   347,   348,   362,   431,   442,   475,  1625,  1865,
        1964,  1965,  2196,  2394,  2395,  2413,  2921,  3324,  3386,
        3496,  3691,  3910,  3934,  4250,  4669,  5410,  5412,  5443,
        5519,  6227,  6260,  7287,  8396, 10266, 10782, 12578, 13467,
       14626, 14785, 14854, 14989, 15195, 15203, 15273, 15486, 15756,
       16080, 16094, 16095, 16132, 16168, 16197, 17621, 17642, 18246,
       18260, 18443, 18734, 18956, 19017, 19066, 20370, 20447, 20953,
       21154, 21180, 21303, 21376, 21386, 22427, 23530, 23923, 23937,
       24048, 24659, 25150, 25238, 25559, 28922, 29176, 29356, 29668,
       29990, 30158, 30870, 31394, 33084, 33227, 33468, 33765, 34512,
       35525, 37186, 37638, 37688, 42473, 43785, 43988, 44277, 44308,
       45293, 46199, 46566, 47078, 49433, 49434, 49764, 49766, 50253,
       51153, 51166, 51272, 51497, 51500, 51756, 51835, 51838, 51954,
       52032, 52620, 52662, 52717, 53118, 53122, 53209, 53487, 55509,
       55642, 55767,

In [65]:
item_lookup.loc[item_lookup.product_id.isin([  343,   347,   348,   362,   431,   442,   475,  1625,  1865,
        1964,  1965,  2196,  2394,  2395,  2413,  2921,  3324,  3386,
        3496,  3691,  3910,  3934,  4250,  4669,  5410,  5412,  5443,
        5519,  6227,  6260,  7287,  8396, 10266, 10782, 12578, 13467,
       14626, 14785, 14854, 14989, 15195, 15203, 15273, 15486, 15756,
       16080, 16094, 16095, 16132, 16168, 16197, 17621, 17642, 18246,
       18260, 18443, 18734, 18956, 19017, 19066, 20370, 20447, 20953,
       21154, 21180, 21303, 21376, 21386, 22427, 23530, 23923, 23937,
       24048, 24659, 25150, 25238, 25559, 28922, 29176, 29356, 29668,
       29990, 30158, 30870, 31394, 33084, 33227, 33468, 33765, 34512,
       35525, 37186, 37638, 37688, 42473, 43785, 43988, 44277, 44308,
       45293, 46199, 46566, 47078, 49433, 49434, 49764, 49766, 50253,
       51153, 51166, 51272, 51497, 51500, 51756, 51835, 51838, 51954,
       52032, 52620, 52662, 52717, 53118, 53122, 53209, 53487, 55509,
       55642, 55767, 55854, 56338, 56343, 56531, 56533, 56631, 56651,
       57120, 57604, 57905, 57913, 57914, 57915])]

,product_id,brand_name


In [69]:
item_lookup[item_lookup.product_id == 343]

,product_id,brand_name
